In [717]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import matplotlib.cm as cm
import math
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from numpy import isnan
import bootstrapped.bootstrap as bs
import random
import bootstrapped.stats_functions as bs_stats
import time
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
import geopandas
import plotly.graph_objects as go
import plotly.figure_factory as ff
import time
from datetime import datetime, date, time, timedelta
from matplotlib import style
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
style.use('ggplot')
from sklearn.svm import NuSVC
import seaborn as sns
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
import warnings
from statistics import mean
warnings.filterwarnings('ignore')

In [698]:
def generate_words(data):
    words = []
    # data(str): csv on show info
        # eg: Bachelor_S22_2018.csv

    df = pd.read_csv(r"/Users/lindseytam/Desktop/team-shows/show_info/"+data) 

    for i in range(len(df["twitter_handle"])):
      
        words.append(df["twitter_handle"][i])
    
    return words

def hidden_key(og_data, new_data, winner):
    #this function lets us know the values for everyone
    #data (str): Bachelor_2018.csv
    contestants = generate_words(og_data) 
    #df = pd.read_csv(r"/Users/lindseytam/Desktop/team-shows/cleaned_data/"+og_data) 
    #df = df.drop(columns=['Unnamed: 0'])
    
    contestant_tweetTotal = []
    winner_data = []
    #contestant_data = []
    for contestant in contestants:
        
        #contestant_tweetTotal.append([contestant, contestant_data["polarity"].mean(), len(contestant_data)])
        if contestant == winner:
            contestant_data = new_data[new_data['text'].str.contains(contestant)]
            if isnan(contestant_data["polarity"].mean()):
                contestant_polarity = 0
            else:
                contestant_polarity = contestant_data["polarity"].mean()
            winner_data.append([contestant_polarity, len(contestant_data)])
        return winner_data
        
def key(og_data, new_data):
    #this function lets us know the values for everyone
    #og_data (str): Bachelor_2018.csv
    contestants = generate_words(og_data) 
    #df = pd.read_csv(r"/Users/lindseytam/Desktop/team-shows/cleaned_data/"+data) 
    #df = df.drop(columns=['Unnamed: 0'])
    
    contestant_tweetTotal = []
    for contestant in contestants:
        contestant_data = new_data[new_data['text'].str.contains(contestant)]
        if isnan(contestant_data["polarity"].mean()):
            contestant_polarity = 0
        else:
            contestant_polarity = contestant_data["polarity"].mean()
        contestant_tweetTotal.append([contestant_polarity, len(contestant_data)])
        
    return contestant_tweetTotal

def clean_tweets(data):
    # just making sure all dates are in datetime
    data = pd.read_csv(r"/Users/lindseytam/Desktop/team-shows/cleaned_data/"+data+".csv") 
    
    data["date"] = pd.to_datetime(df["date"]).dt.date
    data = data.dropna()
    return data

def make_data(data, season, winner, samples):
    # data(str): "Bachelor_2018.csv"
    # season(int): 22
    # winner(str): "thebkoof"
    # samples(int): how many times to bootstrap
    
    open_data = pd.read_csv(r"/Users/lindseytam/Desktop/team-shows/cleaned_data/"+data) 
    
    for i in range(samples):
        open_data = open_data.sample(n=len(open_data),replace=True) #bootstrap the data
        # should n be dependent on length of data??
        # what if winner is not included in bootstrap
        winner_info = hidden_key(data, open_data, winner)
        new_data = key(data, open_data)
        add_empty = 30 - len(new_data)
        for i in range(add_empty):
            new_data.append([0,0])
        np.random.shuffle(new_data)
        flat_list = [item for items in new_data for item in items]
        if not winner_info:
            break
        winner_id = flat_list.index(winner_info[0][0])
        winner_id = winner_id/2
        winner_id = round(winner_id)
        flat_list.append(season) 
        flat_list.append(winner_id)
        globaldata.loc[len(globaldata)] = flat_list
        
    return globaldata
    
def generate_data(samples):
    # samples(int): how many times to bs
    #globaldata = initialize_globaldf()
    
    data = make_data("Bachelor_2018.csv", 22, "thebkoof", samples)
    data1 = make_data("Bachelor_2019.csv", 23, "CassieRandolph", samples)
    data3 = make_data("Bachelorette_2018.csv", 14, "", samples)
    data4 = make_data("Bachelorette_2019.csv", 15, "WyattJed", samples)
    
    inputs = globaldata.drop(columns=['n_season', 'id_winner'])
    outputs = globaldata[["id_winner"]]
                    
    X_train, X_test, Y_train, Y_test = train_test_split(inputs, outputs, test_size=0.2)
    return X_train, X_test, Y_train, Y_test

def initialize_globaldf():
    colname = []
    for i in range(30):
        colname.append("contestant_" + str(i) + "_avg_sentiment")
        colname.append("contestant_" + str(i) + "_total_tweets")
    colname.append("n_season")
    colname.append("id_winner")
    globaldata = pd.DataFrame(columns=colname)
    
    return globaldata

In [702]:
globaldata= initialize_globaldf()
t, c, v, b = generate_data(2)
t

,contestant_0_avg_sentiment,contestant_0_total_tweets,contestant_1_avg_sentiment,contestant_1_total_tweets,contestant_2_avg_sentiment,contestant_2_total_tweets,contestant_3_avg_sentiment,contestant_3_total_tweets,contestant_4_avg_sentiment,contestant_4_total_tweets,...,contestant_25_avg_sentiment,contestant_25_total_tweets,contestant_26_avg_sentiment,contestant_26_total_tweets,contestant_27_avg_sentiment,contestant_27_total_tweets,contestant_28_avg_sentiment,contestant_28_total_tweets,contestant_29_avg_sentiment,contestant_29_total_tweets
0,0.223513,182.0,0.296883,49.0,0.000000,0.0,0.255395,77.0,-0.450000,1.0,...,0.00,0.0,0.277778,1.0,0.00,0.0,0.142590,6.0,0.000000,0.0
5,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.25,4.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0
1,0.277778,1.0,0.225000,1.0,0.243588,180.0,0.000000,2.0,0.000000,3.0,...,0.00,0.0,0.000000,0.0,-0.45,3.0,0.000000,0.0,0.000000,0.0
4,0.000000,0.0,0.089722,36.0,0.000000,0.0,0.000000,0.0,0.043336,260.0,...,0.00,0.0,0.000000,1.0,0.00,0.0,0.084667,160.0,0.237698,9.0


In [658]:
start = 20180101 #show airs
end = 20180313


df = pd.read_csv(r"/Users/lindseytam/Desktop/team-shows/cleaned_data/Bachelor_2018.csv") 
df = df.drop(columns=['Unnamed: 0'])
#y = time.strptime(df["date"][0], "%a %b %d %H:%M:%S +0000 %Y")
#z = time.strptime(df["date"][0], "%a %b %d %H:%M:%S +0000 %Y")

df = clean_tweets("Bachelor_2018")


c = (pd.to_datetime(str(start), format='%Y%m%d') + timedelta(days=7))

# Initialize global dataframe

In [655]:
colname = []
for i in range(30):
    colname.append("contestant_" + str(i) + "_avg_sentiment")
    colname.append("contestant_" + str(i) + "_total_tweets")
colname.append("n_season")
colname.append("id_winner")

globaldata = pd.DataFrame(columns=colname)

# Generate Dataset

In [625]:
og_data = ["Bachelor_2018.csv", "Bachelorette_2019.csv", "Bachelorette_2018.csv", "Bachelorette_2019.csv"]
season = [22, 23, 14, 15]
winner = ["thebkoof", "CassieRandolph", "", "WyattJed"]

In [659]:
y = generate_data(2)

,contestant_0_avg_sentiment,contestant_0_total_tweets,contestant_1_avg_sentiment,contestant_1_total_tweets,contestant_2_avg_sentiment,contestant_2_total_tweets,contestant_3_avg_sentiment,contestant_3_total_tweets,contestant_4_avg_sentiment,contestant_4_total_tweets,...,contestant_26_avg_sentiment,contestant_26_total_tweets,contestant_27_avg_sentiment,contestant_27_total_tweets,contestant_28_avg_sentiment,contestant_28_total_tweets,contestant_29_avg_sentiment,contestant_29_total_tweets,n_season,id_winner
0,0.000000,0.0,0.00000,0.0,0.231039,23.0,0.200000,1.0,0.018571,7.0,...,0.000000,2.0,0.000000,0.0,0.148442,67.0,-0.308611,15.0,22.0,10.0
1,0.232153,177.0,0.00000,0.0,0.124585,98.0,0.175258,265.0,0.219288,62.0,...,0.177436,69.0,0.174269,19.0,0.128064,461.0,0.200000,1.0,22.0,3.0
2,0.000000,0.0,0.00000,1.0,0.000000,0.0,-0.100000,3.0,0.000000,0.0,...,0.170359,43.0,0.000000,0.0,0.000000,0.0,0.220000,5.0,23.0,22.0
3,0.330000,5.0,0.19625,20.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.415451,6.0,0.000000,0.0,0.118446,224.0,0.000000,0.0,23.0,28.0
4,0.297024,6.0,0.00000,0.0,0.000000,3.0,0.000000,1.0,0.000000,0.0,...,0.428571,3.0,0.000000,0.0,0.208081,11.0,0.202900,34.0,15.0,18.0
5,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,9.0,0.500000,2.0,...,0.000000,4.0,0.163487,38.0,0.470370,9.0,0.171929,157.0,15.0,4.0


In [629]:
samples = 100

data = make_data("Bachelor_2018.csv", 22, "thebkoof", samples)
data1 = make_data("Bachelor_2019.csv", 23, "CassieRandolph", samples)
data3 = make_data("Bachelorette_2018.csv", 14, "", samples)
data4 = make_data("Bachelorette_2019.csv", 15, "WyattJed", samples)
    
globaldata

,contestant_0_avg_sentiment,contestant_0_total_tweets,contestant_1_avg_sentiment,contestant_1_total_tweets,contestant_2_avg_sentiment,contestant_2_total_tweets,contestant_3_avg_sentiment,contestant_3_total_tweets,contestant_4_avg_sentiment,contestant_4_total_tweets,...,contestant_26_avg_sentiment,contestant_26_total_tweets,contestant_27_avg_sentiment,contestant_27_total_tweets,contestant_28_avg_sentiment,contestant_28_total_tweets,contestant_29_avg_sentiment,contestant_29_total_tweets,n_season,id_winner
0,0.016667,3.0,0.233334,44.0,0.124836,89.0,0.000000,0.0,0.000000,0.0,...,0.168426,54.0,0.184368,324.0,0.000000,0.0,0.116667,3.0,22.0,27.0
1,0.000000,0.0,-0.150000,3.0,0.000000,0.0,0.116011,61.0,0.237321,135.0,...,-0.450000,1.0,0.022222,3.0,0.000000,0.0,0.000000,0.0,22.0,15.0
2,0.000000,0.0,0.216867,131.0,0.247138,333.0,0.000000,0.0,0.000000,1.0,...,-0.209375,4.0,0.146089,53.0,0.000000,0.0,0.000000,0.0,22.0,2.0
3,0.000000,0.0,0.000000,0.0,0.244204,39.0,0.266306,318.0,0.209963,36.0,...,0.200000,3.0,0.000000,0.0,0.000000,0.0,0.146707,506.0,22.0,3.0
4,0.274038,26.0,0.000000,0.0,0.000000,0.0,0.324226,35.0,0.261915,77.0,...,0.180526,60.0,0.230313,187.0,-0.106250,4.0,0.000000,0.0,22.0,10.0
5,-0.312500,2.0,-0.056000,10.0,0.172197,72.0,0.000000,0.0,0.174304,97.0,...,0.000000,0.0,0.155668,121.0,0.301471,34.0,0.200000,1.0,22.0,8.0
6,-0.312500,3.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.115383,531.0,0.219927,71.0,-0.150000,10.0,22.0,12.0
7,-0.150000,9.0,0.000000,0.0,-0.312500,2.0,0.038333,60.0,0.000000,0.0,...,0.132654,548.0,0.203188,183.0,0.000000,0.0,0.000000,0.0,22.0,19.0
8,0.000000,0.0,-0.312500,1.0,0.000000,0.0,0.000000,0.0,0.341024,92.0,...,0.185974,65.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,22.0,25.0
9,0.223214,42.0,0.019048,7.0,0.150841,569.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,22.0,8.0


In [630]:
data = make_data("Bachelor_2019.csv", 23, "CassieRandolph", 10)

# Regression Time

In [668]:
a, b, c, d = generate_data(3)
a

,contestant_0_avg_sentiment,contestant_0_total_tweets,contestant_1_avg_sentiment,contestant_1_total_tweets,contestant_2_avg_sentiment,contestant_2_total_tweets,contestant_3_avg_sentiment,contestant_3_total_tweets,contestant_4_avg_sentiment,contestant_4_total_tweets,...,contestant_25_avg_sentiment,contestant_25_total_tweets,contestant_26_avg_sentiment,contestant_26_total_tweets,contestant_27_avg_sentiment,contestant_27_total_tweets,contestant_28_avg_sentiment,contestant_28_total_tweets,contestant_29_avg_sentiment,contestant_29_total_tweets
21,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.227573,6.0,...,-0.069444,22.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,5.0
25,-0.500000,2.0,0.000000,0.0,0.000000,0.0,1.000000,1.0,0.037500,8.0,...,0.156250,4.0,0.000000,6.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
19,0.000000,0.0,0.000000,0.0,0.238102,76.0,0.000000,0.0,0.000000,0.0,...,0.225000,5.0,0.000000,0.0,0.208714,25.0,0.000000,0.0,0.203405,308.0
4,0.297024,6.0,0.000000,0.0,0.000000,3.0,0.000000,1.0,0.000000,0.0,...,0.000000,0.0,0.428571,3.0,0.000000,0.0,0.208081,11.0,0.202900,34.0
31,0.000000,0.0,0.000000,0.0,0.173370,311.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.016667,12.0
11,0.306250,16.0,0.000000,4.0,0.000000,0.0,0.225000,12.0,0.345323,45.0,...,0.184599,79.0,-0.177083,10.0,0.000000,0.0,0.163889,19.0,0.000000,0.0
13,0.128334,211.0,0.178101,291.0,0.000000,0.0,0.227804,26.0,0.000000,0.0,...,0.227273,10.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
22,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.183858,43.0,...,0.000000,0.0,0.000000,0.0,0.110898,299.0,0.000000,0.0,0.136364,1.0
26,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,6.0
34,-0.001587,21.0,0.000000,0.0,0.000000,0.0,0.000000,5.0,-0.150000,1.0,...,0.000000,2.0,-0.116667,4.0,0.216204,27.0,0.000000,0.0,0.000000,0.0


## Sample size (n) versus Accuracy

In [713]:
ns=[ 2**b for b in range(3,4) ]
d=60
c=30

scores={ 
    'nb_ave':[],
    'nb_std':[], 
    'lda_ave':[], 
    'lda_std':[], 
    'lr_ave':[], 
    'lr_std':[],
    'lrcv_ave':[], 
    'lrcv_std':[]
}

for n in ns:
    
    scores_itr={
    'nb':[], 
    'lda': [], 
    'lr':[]
    }
    
    for trial in range(5):
        globaldata= initialize_globaldf()
        X_train, X_test, Y_train, Y_test=generate_data(n)
        
        # train a naive bayes model
        nb = GaussianNB()
        nb.fit(X_train,Y_train) 
        scores_itr['nb'].append(nb.score(X_test,Y_test))
        
        # train a LogisticRegressionCV
        lrcv = LogisticRegressionCV()
        lrcv.fit(X_train,Y_train) 
        scores_itr['lrcv'].append(lrcv.score(X_test,Y_test))
        
        # train lda
        lda = LinearDiscriminantAnalysis() 
        lda.fit(X_train,Y_train) 
        scores_itr['lda'].append(lda.score(X_test,Y_test))
        
        # train lr
        lr = LogisticRegression() 
        lr.fit(X_train,Y_train) 
        scores_itr['lr'].append(lr.score(X_test,Y_test))
    
    # record the mean and standard deviations of each experiment
    scores['nb_ave'].append(np.mean(scores_itr['nb'])) 
    scores['nb_std'].append(np.std(scores_itr['nb']))
    
    scores['lrcv_ave'].append(np.mean(scores_itr['lrcv'])) 
    scores['lrcv_std'].append(np.std(scores_itr['lrcv']))
    
    scores['lda_ave'].append(np.mean(scores_itr['lda'])) 
    scores['lda_std'].append(np.std(scores_itr['lda']))
    
    scores['lr_ave'].append(np.mean(scores_itr['lr'])) 
    scores['lr_std'].append(np.std(scores_itr['lr']))
    
    
# plot the results of each experiment
plt.figure(figsize=(30,40))
plt.errorbar(ns, scores['nb_ave'], yerr=scores['nb_std'], label='naive bayes')
plt.errorbar(ns, scores['lda_ave'], yerr=scores['lda_std'], label='lda')
plt.errorbar(ns, scores['lr_ave'], yerr=scores['lr_std'], label='lr' )
plt.errorbar(ns, scores['lrcv_ave'], yerr=scores['lrcv_std'], label='lrcv' )
plt.xscale('log',basex=2) 
plt.xlabel('n') 
plt.ylabel('classification accuracy') 
plt.legend() 
plt.savefig("/Users/lindseytam/Desktop/team-shows/Shiny/www/prediction1.png")
plt.show()

KeyboardInterrupt: 

In [ ]:
ns=[ 2**b for b in range(3,15) ]
d=60
c=30

scores={ 
    'svc_ave':[],
    'svc_std':[], 
    'lrcv_ave':[], 
    'lrcv_std':[], 
    'sgd_ave':[], 
    'sgd_std':[],
}

for n in ns:
    
    scores_itr={
    'svc':[],  
    'sgd':[],
    'lrcv':[]
    }
    
    for trial in range(5):
        globaldata= initialize_globaldf()
        X_train, X_test, Y_train, Y_test=generate_data(n)
        
        # train a LinearSVC model
        svc = LinearSVC()
        svc.fit(X_train,Y_train) 
        scores_itr['svc'].append(svc.score(X_test,Y_test))
        
        # train a LogisticRegressionCV
        #lrcv = LogisticRegressionCV()
        #lrcv.fit(X_train,Y_train) 
        #scores_itr['lrcv'].append(lrcv.score(X_test,Y_test))
        
        # train lda
        sgd = linear_model.SGDClassifier() 
        sgd.fit(X_train,Y_train) 
        scores_itr['sgd'].append(sgd.score(X_test,Y_test))
        
    
    # record the mean and standard deviations of each experiment
    scores['svc_ave'].append(np.mean(scores_itr['svc'])) 
    scores['svc_std'].append(np.std(scores_itr['svc']))
    
    
    scores['sgd_ave'].append(np.mean(scores_itr['sgd'])) 
    scores['sgd_std'].append(np.std(scores_itr['sgd']))
    
    
    #scores['lrcv_ave'].append(np.mean(scores_itr['lrcv'])) 
    #scores['lrcv_std'].append(np.std(scores_itr['lrcv']))
    
    
# plot the results of each experiment
plt.figure(figsize=(20,10))
plt.errorbar(ns, scores['svc_ave'], yerr=scores['svc_std'], label='LinearSVC')
plt.errorbar(ns, scores['sgd_ave'], yerr=scores['sgd_std'], label='sgd' )
#plt.errorbar(ns, scores['lrcv_ave'], yerr=scores['lrcv_std'], label='lrcv' )
plt.xscale('log',basex=2) 
plt.xlabel('n') 
plt.ylabel('classification accuracy') 
plt.legend() 
plt.show()